In [1]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.model_selection import train_test_split
import torch
import torch.nn as nn
import torch.nn.functional as F
from model_architecture import PumpClassifier
from tqdm import tqdm

In [2]:
df=pd.read_csv('data.csv')
df.columns=['col1','col2','col3','col4','target']

In [3]:
x=df[['col1','col2','col3','col4']].copy()
y=df['target']

In [4]:
scaler=StandardScaler()
x[['col1','col2']]=scaler.fit_transform(x[['col1','col2']])
x['col3']=(x['col3']==130).astype(int)

In [5]:
le=LabelEncoder()
y=le.fit_transform(y)

In [6]:
X_train,X_test,y_train,y_test=train_test_split(x.values,y,test_size=0.2,random_state=42)
X_train_tensor=torch.tensor(X_train,dtype=torch.float32)
X_test_tensor=torch.tensor(X_test,dtype=torch.float32)
y_train_tensor=torch.tensor(y_train,dtype=torch.long)
y_test_tensor=torch.tensor(y_test,dtype=torch.long)

In [7]:
model=PumpClassifier()
criterion=nn.CrossEntropyLoss()
optimizer=torch.optim.Adam(model.parameters(),lr=0.001)

In [12]:
n_epochs=100
for epoch in range(n_epochs):
  model.train()
  optimizer.zero_grad()
  output=model(X_train_tensor)
  loss=criterion(output,y_train_tensor)
  loss.backward()
  optimizer.step()

  model.eval()
  with torch.no_grad():
    test_output=model(X_test_tensor)
    test_pred=torch.argmax(test_output,dim=1)
    test_acc=(test_pred==y_test_tensor).float().mean()
  
  if epoch %5==0 or epoch==n_epochs-1:
    print(f"Epoch{epoch:>2} | Train Loss: {loss.item():.4f} | Test Acc:{test_acc.item():.2%}")

Epoch 0 | Train Loss: 0.2580 | Test Acc:94.12%
Epoch 5 | Train Loss: 0.2454 | Test Acc:95.29%
Epoch10 | Train Loss: 0.2342 | Test Acc:96.47%
Epoch15 | Train Loss: 0.2222 | Test Acc:96.47%
Epoch20 | Train Loss: 0.2119 | Test Acc:96.47%
Epoch25 | Train Loss: 0.2014 | Test Acc:96.47%
Epoch30 | Train Loss: 0.1929 | Test Acc:96.47%
Epoch35 | Train Loss: 0.1831 | Test Acc:96.47%
Epoch40 | Train Loss: 0.1751 | Test Acc:96.47%
Epoch45 | Train Loss: 0.1670 | Test Acc:96.47%
Epoch50 | Train Loss: 0.1597 | Test Acc:96.47%
Epoch55 | Train Loss: 0.1530 | Test Acc:96.47%
Epoch60 | Train Loss: 0.1465 | Test Acc:96.47%
Epoch65 | Train Loss: 0.1396 | Test Acc:96.47%
Epoch70 | Train Loss: 0.1339 | Test Acc:96.47%
Epoch75 | Train Loss: 0.1281 | Test Acc:96.47%
Epoch80 | Train Loss: 0.1233 | Test Acc:96.47%
Epoch85 | Train Loss: 0.1178 | Test Acc:97.65%
Epoch90 | Train Loss: 0.1130 | Test Acc:97.65%
Epoch95 | Train Loss: 0.1084 | Test Acc:97.65%
Epoch99 | Train Loss: 0.1054 | Test Acc:97.65%


In [13]:
torch.save(model.state_dict(),'pumpPredictionModel.pt')

import joblib
joblib.dump(scaler,'scaler.save')
joblib.dump(le,'label_encoder.save')

['label_encoder.save']